In [1]:
from multidoc2_chat_exp_init import *

In [2]:
inference_model = model_init()

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForCausalLM were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.encoder.layer.9.crossattention.self.query.bias', 'roberta.encoder.layer.0.crossattention.output.LayerNorm.bias', 'roberta.encoder.layer.0.crossattention.self.quer



Num Params: 277716061


In [11]:
state_dict = torch.load('trained_models/first_attempt_robert2roberta_/checkpoint-26820/pytorch_model.bin')
inference_model.load_state_dict(state_dict)

<All keys matched successfully>

In [12]:
inference_dataset = MultiDoc2ChatDataset(tokenizer,data_pack=[])
inference_dataset.change_data_mode(0)

In [13]:
chat_data = dict(facts="If a value of 4 (dry) or more, or an increase of more than 1 (dry) from the previous month's colour rating is obtained, a double membrane test should be performed as a first step in an investigation.".replace('\n',''),
                 utterance="",
                 current_question="What is the maximum color rating?",
                 conv_history= ""
                 
                 )


chat_data = dict(
    facts="Because we see customers make these mistakes over and over again , we are issuing this list of the top five DMV mistakes and how to avoid them. Forgetting to Update Address By statute , you must report a change of address to DMV within ten days of moving. That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. It is not sufficient to only: write your new address on the back of your old license;",

    current_question ="Hello, I forgot to update my address, can you help me with that?",
    conv_history= "",
    utterance="",
)
chat_data = dict(
    conv_history= "",
    utterance="",
    current_question ="Can you tell me a little about AFCON?",
    facts="The Africa Cup of Nations referred to as AFCON (French: Coupe d'Afrique des Nations, sometimes referred to as CAN, or TotalEnergies Africa Cup of Nations for sponsorship reasons), and sometimes as African Cup of Nations, is the main international men's association football competition in Africa. It is sanctioned by the Confederation of African Football (CAF), and was first held in 1957. Since 1968, it has been held every two years, switching to odd-numbered years in 2013."
)

input_data = inference_dataset.processExample(chat_data)



In [14]:
input_ids = pad_seq(input_data.input_ids,400,tokenizer.pad_token_id).view(1,-1).to(inference_model.device)
attention_mask = pad_seq(input_data.attention_mask,400,tokenizer.pad_token_id).view(1,-1).to(inference_model.device)
decoder_inputs = input_data.labels.view(1,-1).to(inference_model.device)

In [15]:
response = inference_model.generate(
    input_ids,
    #attention_mask=attention_mask,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
    length_penalty=1.25,
    early_stopping=True,
    use_cache=True,
    num_beams=4,
    no_repeat_ngram_size=2,
    repetition_penalty=1.56,
)

tokenizer.decode(response[0])

'<s><s>[RESPONSE] Gladly. The ACA is the fastest way to get a CDL, and it can be done through the helpline.</s>'

'<s><s>[RESPONSE] Of course. You must report a change of address to DMV within ten days of moving.</s>'